<a href="https://colab.research.google.com/github/MoqiSheng/MoqiSheng.github.io/blob/main/prepare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/UrbanASIF/plus/anchor_Shenzhen_plus.zip -d /content/anchor

Archive:  /content/drive/MyDrive/UrbanASIF/plus/anchor_Shenzhen_plus.zip
  inflating: /content/anchor/0.png   
  inflating: /content/anchor/1.png   
  inflating: /content/anchor/100.png  
  inflating: /content/anchor/1021.png  
  inflating: /content/anchor/1028.png  
  inflating: /content/anchor/1035.png  
  inflating: /content/anchor/1056.png  
  inflating: /content/anchor/106.png  
  inflating: /content/anchor/1062.png  
  inflating: /content/anchor/107.png  
  inflating: /content/anchor/1072.png  
  inflating: /content/anchor/1073.png  
  inflating: /content/anchor/108.png  
  inflating: /content/anchor/1087.png  
  inflating: /content/anchor/1110.png  
  inflating: /content/anchor/1111.png  
  inflating: /content/anchor/1135.png  
  inflating: /content/anchor/1166.png  
  inflating: /content/anchor/1167.png  
  inflating: /content/anchor/1172.png  
  inflating: /content/anchor/1175.png  
  inflating: /content/anchor/118.png  
  inflating: /content/anchor/1180.png  
  inflating: /co

In [3]:
pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-9gshonty
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-9gshonty
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.2 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=8423f09eec1a8a309158c75d70ca468fb870e451cffcedb3958476f2fd856bb3
  Stored in directory: /tmp/pip-ephem-wheel-cache-gnovf59b/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [5]:
import os
import torch
import clip
from PIL import Image
from tqdm import tqdm

# 加载预训练的 CLIP 模型
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14@336px", device=device) # model 加载 CLIP 模型的预训练版本 ViT-L/14@336px；preprocess 是图像的预处理函数，用于将原始图像转化为符合模型的要求的输入

# 定义特征提取函数 extract_features
def extract_features(image_folder, output_file):
    image_features_list = []
    # 读取 image_folder 中所有扩展名为 .jpg 或 .png 的文件，并将文件名按自然数顺序排序（例如，1.jpg、2.jpg、3.jpg 顺序排列）
    image_paths = sorted(
        [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.lower().endswith(('.jpg', '.png'))],
        key=lambda x: int(os.path.splitext(os.path.basename(x))[0])
    )

    for image_path in tqdm(image_paths, desc=f"Processing {image_folder}"):
        try:
            image = preprocess(Image.open(image_path)).unsqueeze(0).to(device) # preprocess 函数对图像进行预处理
            with torch.no_grad(): # 禁用了梯度计算（不需要反向传播）
                image_features = model.encode_image(image) # 使用 CLIP 模型的 encode_image 方法对图像进行编码，返回图像的特征向量
                image_features /= image_features.norm(dim=-1, keepdim=True) # 归一化处理，image_features 除以它的模长将其归一化
                image_features_list.append(image_features.cpu()) # 将特征向量移动到 CPU ，然后将其追加到列表 image_features_list 中，以便后续保存。
        except Exception as e:
            print(f"Error processing {image_path}: {e}")

    # 将所有图像特征保存为矩阵
    if image_features_list:
        image_features = torch.cat(image_features_list, dim=0) # 将存储在 image_features_list 中的所有图像特征向量[ n ]沿着行向拼接成一个大的张量[m , n]（tensor）
        torch.save(image_features, output_file)
        print(f"Features saved to {output_file}")
    else:
        print(f"No valid images found in {image_folder}")

# 调用特征提取函数，提取并保存每个城市的特征
extract_features('./anchor', './imgs_clip_anchor_plus')

Processing ./anchor: 100%|██████████| 240/240 [00:06<00:00, 35.97it/s]

Features saved to ./imgs_clip_anchor_plus


In [6]:
!unzip /content/drive/MyDrive/UrbanASIF/plus/predict_Shenzhen_plus.zip -d /content/predict

Archive:  /content/drive/MyDrive/UrbanASIF/plus/predict_Shenzhen_plus.zip
  inflating: /content/predict/10.png  
  inflating: /content/predict/1000.png  
  inflating: /content/predict/1001.png  
  inflating: /content/predict/1002.png  
  inflating: /content/predict/1003.png  
  inflating: /content/predict/1004.png  
  inflating: /content/predict/1005.png  
  inflating: /content/predict/1006.png  
  inflating: /content/predict/1007.png  
  inflating: /content/predict/1008.png  
  inflating: /content/predict/1009.png  
  inflating: /content/predict/101.png  
  inflating: /content/predict/1010.png  
  inflating: /content/predict/1011.png  
  inflating: /content/predict/1012.png  
  inflating: /content/predict/1013.png  
  inflating: /content/predict/1014.png  
  inflating: /content/predict/1015.png  
  inflating: /content/predict/1016.png  
  inflating: /content/predict/1017.png  
  inflating: /content/predict/1018.png  
  inflating: /content/predict/1019.png  
  inflating: /content/predi

In [7]:
import os
import torch
import clip
from PIL import Image
from tqdm import tqdm

# 加载预训练的 CLIP 模型
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14@336px", device=device) # model 加载 CLIP 模型的预训练版本 ViT-L/14@336px；preprocess 是图像的预处理函数，用于将原始图像转化为符合模型的要求的输入

# 定义特征提取函数 extract_features
def extract_features(image_folder, output_file):
    image_features_list = []
    # 读取 image_folder 中所有扩展名为 .jpg 或 .png 的文件，并将文件名按自然数顺序排序（例如，1.jpg、2.jpg、3.jpg 顺序排列）
    image_paths = sorted(
        [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.lower().endswith(('.jpg', '.png'))],
        key=lambda x: int(os.path.splitext(os.path.basename(x))[0])
    )

    for image_path in tqdm(image_paths, desc=f"Processing {image_folder}"):
        try:
            image = preprocess(Image.open(image_path)).unsqueeze(0).to(device) # preprocess 函数对图像进行预处理
            with torch.no_grad(): # 禁用了梯度计算（不需要反向传播）
                image_features = model.encode_image(image) # 使用 CLIP 模型的 encode_image 方法对图像进行编码，返回图像的特征向量
                image_features /= image_features.norm(dim=-1, keepdim=True) # 归一化处理，image_features 除以它的模长将其归一化
                image_features_list.append(image_features.cpu()) # 将特征向量移动到 CPU ，然后将其追加到列表 image_features_list 中，以便后续保存。
        except Exception as e:
            print(f"Error processing {image_path}: {e}")

    # 将所有图像特征保存为矩阵
    if image_features_list:
        image_features = torch.cat(image_features_list, dim=0) # 将存储在 image_features_list 中的所有图像特征向量[ n ]沿着行向拼接成一个大的张量[m , n]（tensor）
        torch.save(image_features, output_file)
        print(f"Features saved to {output_file}")
    else:
        print(f"No valid images found in {image_folder}")

# 调用特征提取函数，提取并保存每个城市的特征
extract_features('./predict', './imgs_clip_predict_plus')

Processing ./predict: 100%|██████████| 1278/1278 [00:35<00:00, 35.93it/s]

Features saved to ./imgs_clip_predict_plus


In [8]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.preprocessing import normalize
import random  # 导入 random 模块
import numpy as np

# 设置随机种子，确保结果可复现
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # 如果你使用了GPU，设置所有GPU的种子
torch.backends.cudnn.deterministic = True  # 确保CuDNN确定性
torch.backends.cudnn.benchmark = False  # 禁用CuDNN的自动优化以确保可复现性

# 加载预训练的 Sentence Transformer 模型
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# urbanclip 模板
urbanclip_templates = [
    'A street photo of {} with cars.',
    'A street photo of {} with parking lot.',
    'A street photo of {} on the road.',
    'A street photo of {} with many trees.',
    'A street photo of {} in city.'
]

# 读取数据
df = pd.read_csv('./anchor_Shenzhen_plus.csv')

# 用于存储每个嵌入
embeddings = []

# 遍历每一行
for index, row in df.iterrows():
    primary_index = row['primary_index']

    # 随机选取一个模板
    template = random.choice(urbanclip_templates)

    # 生成句子
    sentence = template.format(primary_index)

    # 编码句子，得到嵌入
    sentence_embedding = model.encode([sentence])  # 注意这里只编码一个句子

    # 归一化句子的嵌入
    normalized_embedding = normalize(sentence_embedding, axis=1)

    # 转换为 PyTorch tensor
    final_embedding_tensor = torch.tensor(normalized_embedding[0], dtype=torch.float32)

    # 保存嵌入
    embeddings.append(final_embedding_tensor)

# 将嵌入转换为 PyTorch 张量
embedding_tensor = torch.stack(embeddings)  # Shape: [num_samples, embedding_dim]

# 保存到 .pt 文件
torch.save(embedding_tensor, 'txts_anchor_plus')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.preprocessing import normalize
import json

# 加载预训练的 Sentence Transformer 模型
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# urbanclip 模板
urbanclip_templates = [
    '{}',
    'A street photo of {} with cars.',
    'A street photo of {} with parking lot.',
    'A street photo of {} on the road.',
    'A street photo of {} with many trees.',
    'A street photo of {} in city.'
]

# 读取 urban_taxonomy.json
with open('./Utils/urban_taxonomy.json', 'r') as f:
    urban_taxonomy = json.load(f)

# 获取 urban_taxonomy 中所有的小类（例如 "bay", "beach" 等）
all_categories = []
for category in urban_taxonomy.values():
    all_categories.extend(category)

# 用于存储每个嵌入
embeddings = []

# 遍历所有的小类
for category in all_categories:
    # 每个模板生成一个句子
    sentences = [template.format(category) for template in urbanclip_templates]

    # 编码句子，得到嵌入
    sentence_embeddings = model.encode(sentences)

    # 归一化每个句子的嵌入（参考 UrbanCLIP zeroshot.py 的处理思路）
    normalized_embeddings = normalize(sentence_embeddings, axis=1)

    # 计算 6 个句子的归一化嵌入的平均值
    avg_embedding = np.mean(normalized_embeddings, axis=0)

    # 归一化最终的平均嵌入
    final_embedding = normalize([avg_embedding])[0]

    # 转换为 PyTorch tensor
    final_embedding_tensor = torch.tensor(final_embedding, dtype=torch.float32)

    # 保存嵌入
    embeddings.append(final_embedding_tensor)

# 将嵌入转换为 PyTorch 张量
embedding_tensor = torch.stack(embeddings)  # Shape: [num_samples, embedding_dim]

# 保存到 .pt 文件
torch.save(embedding_tensor, 'txts_answer')